In [1]:
import os
import cv2
import glob
import random
import warnings
import numpy as np
from PIL import Image
import disarray as da
import seaborn as sns
import tensorflow as tf
from imutils import paths
from tensorflow.keras.utils import load_img, img_to_array
from itertools import cycle
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [21]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
base_path = '/home/sanjeev/ML_Dataset/microsoft_dataset/asm_img/'

In [3]:
cur_dir = os.getcwd()
os.chdir(base_path)

In [4]:
img_list = glob.glob(os.path.join(base_path, '*/*.png'))

In [5]:
categories = os.listdir(base_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

In [6]:
cur_dir = os.getcwd()
os.chdir(base_path)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family

for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
fam = []
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
        fam.append(list_fams[label])
    label += 1
num_classes = label

Label: 0	Family: .ipynb_checkpoints	Number of images: 0
Label: 1	Family:           Gatak	Number of images: 1013
Label: 2	Family:    Kelihos_ver1	Number of images: 398
Label: 3	Family:    Kelihos_ver3	Number of images: 2942
Label: 4	Family:        Lollipop	Number of images: 2478
Label: 5	Family:  Obfuscator.ACY	Number of images: 1228
Label: 6	Family:          Ramnit	Number of images: 1541
Label: 7	Family:           Simda	Number of images: 42
Label: 8	Family:          Tracur	Number of images: 751
Label: 9	Family:           Vundo	Number of images: 475


In [7]:
asm_features = np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/asm_plots/MobileNetV3_Large_X.npy')
y = np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/asm_plots/MobileNetV3_Large_y.npy')

In [11]:
asm_features.shape

(10868, 1024)

In [12]:
y.shape

(10868,)

In [14]:
bytes_fetaures=np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/bytes_plots/MobileNetV3_Large_X.npy')

In [16]:
bytes_fetaures.shape

(10868, 1024)

## spliting data

In [17]:
X_asm_train,X_asm_test,X_byte_train,X_byte_test,y_train,y_test=train_test_split(asm_features,bytes_fetaures,y,test_size=0.3,random_state=42,stratify=y)

In [18]:

print(f"Training samples: {len(X_asm_train)}")
print(f"Test samples: {len(X_asm_test)}")

Training samples: 7607
Test samples: 3261


## startegy 1:only for asm features ensemble voting weith mobilent v3 large 

In [19]:
print("\n" + "="*50)
print("STRATEGY 1: ASM FEATURES ONLY")
print("="*50)


STRATEGY 1: ASM FEATURES ONLY


In [22]:
asm_scaler = StandardScaler()
X_asm_train_scaled = asm_scaler.fit_transform(X_asm_train)
X_asm_test_scaled = asm_scaler.transform(X_asm_test)

In [25]:
%%time
# Train models on ASM features
rf_asm = RandomForestClassifier(n_estimators=100, random_state=42)
rf_asm.fit(X_asm_train_scaled, y_train)


CPU times: user 13.9 s, sys: 0 ns, total: 13.9 s
Wall time: 13.9 s


RandomForestClassifier(random_state=42)

In [26]:
%%time
rf_asm_pred = rf_asm.predict(X_asm_test_scaled)

CPU times: user 111 ms, sys: 11.2 ms, total: 122 ms
Wall time: 120 ms


In [27]:
rf_asm_pred

array(['Gatak', 'Obfuscator.ACY', 'Ramnit', ..., 'Obfuscator.ACY',
       'Kelihos_ver3', 'Kelihos_ver3'], dtype='<U14')

In [31]:
rf_asm_acc = accuracy_score(y_test, rf_asm_pred)

In [32]:
rf_asm_acc

0.8880711438209138

In [29]:
%%time
print("training Accuracy:", round((rf_asm.score(X_asm_train_scaled, y_train) * 100), 2))

training Accuracy: 100.0
CPU times: user 240 ms, sys: 2.59 ms, total: 242 ms
Wall time: 241 ms


In [30]:
%%time
print("Testing Accuracy:", round((rf_asm.score(X_asm_test_scaled, y_test) * 100), 2))

Testing Accuracy: 88.81
CPU times: user 116 ms, sys: 2.7 ms, total: 119 ms
Wall time: 117 ms


In [33]:
gb_asm = GradientBoostingClassifier(n_estimators=100, random_state=42)

In [34]:
%%time
gb_asm.fit(X_asm_train_scaled, y_train)

CPU times: user 21min 58s, sys: 74.2 ms, total: 21min 58s
Wall time: 21min 58s


GradientBoostingClassifier(random_state=42)

In [35]:
%%time
gb_asm_pred = gb_asm.predict(X_asm_test_scaled)

CPU times: user 81.8 ms, sys: 7.96 ms, total: 89.8 ms
Wall time: 87.4 ms


In [36]:
%%time
gb_asm_acc = accuracy_score(y_test, gb_asm_pred)

CPU times: user 5.83 ms, sys: 11 µs, total: 5.84 ms
Wall time: 4.44 ms


In [37]:
gb_asm_acc

0.8856179086169886

In [40]:
%%time
svm_asm = SVC(kernel='rbf', probability=True, random_state=42)
svm_asm.fit(X_asm_train_scaled, y_train)


CPU times: user 57.5 s, sys: 76 ms, total: 57.6 s
Wall time: 57.6 s


SVC(probability=True, random_state=42)

In [42]:
%%time
svm_asm_pred = svm_asm.predict(X_asm_test_scaled)

CPU times: user 8.5 s, sys: 8.01 ms, total: 8.51 s
Wall time: 8.51 s


In [43]:
%%time
svm_asm_acc = accuracy_score(y_test, svm_asm_pred)

CPU times: user 3.61 ms, sys: 12 µs, total: 3.62 ms
Wall time: 2.31 ms


In [44]:
svm_asm_acc

0.8920576510272923

In [45]:
print(f"Random Forest accuracy: {rf_asm_acc:.4f}")
print(f"Gradient Boosting accuracy: {gb_asm_acc:.4f}")
print(f"SVM accuracy: {svm_asm_acc:.4f}")


Random Forest accuracy: 0.8881
Gradient Boosting accuracy: 0.8856
SVM accuracy: 0.8921


### Ensemble voting for ASM
### Hard voting

In [48]:
%%time
asm_preds = np.column_stack([rf_asm_pred, gb_asm_pred, svm_asm_pred])

CPU times: user 990 µs, sys: 0 ns, total: 990 µs
Wall time: 879 µs


In [49]:
asm_preds

array([['Gatak', 'Gatak', 'Gatak'],
       ['Obfuscator.ACY', 'Obfuscator.ACY', 'Obfuscator.ACY'],
       ['Ramnit', 'Ramnit', 'Ramnit'],
       ...,
       ['Obfuscator.ACY', 'Obfuscator.ACY', 'Obfuscator.ACY'],
       ['Kelihos_ver3', 'Kelihos_ver3', 'Kelihos_ver3'],
       ['Kelihos_ver3', 'Kelihos_ver3', 'Kelihos_ver3']], dtype='<U14')

In [97]:
asm_hard_vote = []
for row in asm_preds:
    unique, counts = np.unique(row, return_counts=True)
    majority_class = unique[np.argmax(counts)]
    asm_hard_vote.append(majority_class)
asm_hard_vote = np.array(asm_hard_vote)
asm_hard_acc = accuracy_score(y_test, asm_hard_vote)


In [98]:
asm_hard_vote

array(['Gatak', 'Obfuscator.ACY', 'Ramnit', ..., 'Obfuscator.ACY',
       'Kelihos_ver3', 'Kelihos_ver3'], dtype='<U14')

### # Soft voting

In [99]:
rf_asm_proba = rf_asm.predict_proba(X_asm_test_scaled)
gb_asm_proba = gb_asm.predict_proba(X_asm_test_scaled)
svm_asm_proba = svm_asm.predict_proba(X_asm_test_scaled)
asm_avg_proba = (rf_asm_proba + gb_asm_proba + svm_asm_proba) / 3
asm_soft_vote = np.argmax(asm_avg_proba, axis=1)
asm_soft_acc = accuracy_score(y_test_fixed, asm_soft_vote)

In [100]:
print(f"Hard Voting accuracy: {asm_hard_acc:.4f}")
print(f"Soft Voting accuracy: {asm_soft_acc:.4f}")

Hard Voting accuracy: 0.8964
Soft Voting accuracy: 0.8970


## bytes voting

In [101]:
print("\n" + "="*50)
print("STRATEGY 2:BYTE FEATURES ONLY")
print("="*50)


STRATEGY 2:BYTE FEATURES ONLY


In [67]:
# Scale byte features
byte_scaler = StandardScaler()
X_byte_train_scaled = byte_scaler.fit_transform(X_byte_train)
X_byte_test_scaled = byte_scaler.transform(X_byte_test)

# Train models on byte features
rf_byte = RandomForestClassifier(n_estimators=100, random_state=42)
rf_byte.fit(X_byte_train_scaled, y_train)
rf_byte_pred = rf_byte.predict(X_byte_test_scaled)
rf_byte_acc = accuracy_score(y_test, rf_byte_pred)

gb_byte = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_byte.fit(X_byte_train_scaled, y_train)
gb_byte_pred = gb_byte.predict(X_byte_test_scaled)
gb_byte_acc = accuracy_score(y_test, gb_byte_pred)

svm_byte = SVC(kernel='rbf', probability=True, random_state=42)
svm_byte.fit(X_byte_train_scaled, y_train)
svm_byte_pred = svm_byte.predict(X_byte_test_scaled)
svm_byte_acc = accuracy_score(y_test, svm_byte_pred)

print(f"Random Forest accuracy: {rf_byte_acc:.4f}")
print(f"Gradient Boosting accuracy: {gb_byte_acc:.4f}")
print(f"SVM accuracy: {svm_byte_acc:.4f}")


Random Forest accuracy: 0.9144
Gradient Boosting accuracy: 0.8767
SVM accuracy: 0.8657


In [92]:
# Ensemble voting for byte features
# Hard voting
byte_preds = np.column_stack([rf_byte_pred, gb_byte_pred, svm_byte_pred])
byte_hard_vote = []
for row in byte_preds:
    unique, counts = np.unique(row, return_counts=True)
    majority_class = unique[np.argmax(counts)]
    byte_hard_vote.append(majority_class)
byte_hard_vote = np.array(byte_hard_vote)
byte_hard_acc = accuracy_score(y_test, byte_hard_vote)

# Soft voting
rf_byte_proba = rf_byte.predict_proba(X_byte_test_scaled)
gb_byte_proba = gb_byte.predict_proba(X_byte_test_scaled)
svm_byte_proba = svm_byte.predict_proba(X_byte_test_scaled)
byte_avg_proba = (rf_byte_proba + gb_byte_proba + svm_byte_proba) / 3
byte_soft_vote = np.argmax(byte_avg_proba, axis=1)
byte_soft_acc = accuracy_score(y_test_fixed, byte_soft_vote)

print(f"Hard Voting accuracy: {byte_hard_acc:.4f}")
print(f"Soft Voting accuracy: {byte_soft_acc:.4f}")


Hard Voting accuracy: 0.8954
Soft Voting accuracy: 0.8942


## stacked


In [71]:
fused_features = np.concatenate([asm_features, bytes_fetaures], axis=1)

In [73]:
print("\n" + "="*50)
print("STRATEGY 3: EARLY FUSION (CONCATENATION)")
print("="*50)

# Scale features separately then concatenate
asm_scaler_fusion = StandardScaler()
byte_scaler_fusion = StandardScaler()

X_asm_train_scaled_fusion = asm_scaler_fusion.fit_transform(X_asm_train)
X_asm_test_scaled_fusion = asm_scaler_fusion.transform(X_asm_test)

X_byte_train_scaled_fusion = byte_scaler_fusion.fit_transform(X_byte_train)
X_byte_test_scaled_fusion = byte_scaler_fusion.transform(X_byte_test)

# Concatenate features
X_train_fused = np.hstack([X_asm_train_scaled_fusion, X_byte_train_scaled_fusion])
X_test_fused = np.hstack([X_asm_test_scaled_fusion, X_byte_test_scaled_fusion])

# Final scaling
fusion_scaler = StandardScaler()
X_train_fused = fusion_scaler.fit_transform(X_train_fused)
X_test_fused = fusion_scaler.transform(X_test_fused)

print(f"Fused features shape: {X_train_fused.shape}")


STRATEGY 3: EARLY FUSION (CONCATENATION)
Fused features shape: (7607, 2048)


In [74]:
rf_fused = RandomForestClassifier(n_estimators=100, random_state=42)
rf_fused.fit(X_train_fused, y_train)
rf_fused_pred = rf_fused.predict(X_test_fused)
rf_fused_acc = accuracy_score(y_test, rf_fused_pred)


In [75]:
rf_fused_acc

0.9408157007053051

In [77]:
%%time
gb_fused = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_fused.fit(X_train_fused, y_train)
gb_fused_pred = gb_fused.predict(X_test_fused)
gb_fused_acc = accuracy_score(y_test, gb_fused_pred)

CPU times: user 44min 36s, sys: 320 ms, total: 44min 36s
Wall time: 44min 37s


In [78]:
gb_fused_acc

0.9310027598896045

In [79]:
%%time
svm_fused = SVC(kernel='rbf', probability=True, random_state=42)
svm_fused.fit(X_train_fused, y_train)
svm_fused_pred = svm_fused.predict(X_test_fused)
svm_fused_acc = accuracy_score(y_test, svm_fused_pred)


CPU times: user 2min 21s, sys: 112 ms, total: 2min 22s
Wall time: 2min 22s


In [80]:
print(f"Random Forest accuracy: {rf_fused_acc:.4f}")
print(f"Gradient Boosting accuracy: {gb_fused_acc:.4f}")
print(f"SVM accuracy: {svm_fused_acc:.4f}")


Random Forest accuracy: 0.9408
Gradient Boosting accuracy: 0.9310
SVM accuracy: 0.9399


In [90]:
# Hard voting
fused_preds = np.column_stack([rf_fused_pred, gb_fused_pred, svm_fused_pred])
fused_hard_vote = []
for row in fused_preds:
    unique, counts = np.unique(row, return_counts=True)
    majority_class = unique[np.argmax(counts)]
    fused_hard_vote.append(majority_class)
fused_hard_vote = np.array(fused_hard_vote)
fused_hard_acc = accuracy_score(y_test, fused_hard_vote)


In [91]:
fused_hard_acc

0.9432689359092303

In [88]:
# Soft voting
rf_fused_proba = rf_fused.predict_proba(X_test_fused)
gb_fused_proba = gb_fused.predict_proba(X_test_fused)
svm_fused_proba = svm_fused.predict_proba(X_test_fused)
fused_avg_proba = (rf_fused_proba + gb_fused_proba + svm_fused_proba) / 3
fused_soft_vote = np.argmax(fused_avg_proba, axis=1)
fused_soft_acc = accuracy_score(y_test_fixed, fused_soft_vote)


In [89]:
fused_soft_acc

0.9466421343146274

In [93]:
print(f"Hard Voting accuracy: {fused_hard_acc:.4f}")
print(f"Soft Voting accuracy: {fused_soft_acc:.4f}")

Hard Voting accuracy: 0.9433
Soft Voting accuracy: 0.9466
